In [3]:
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

train_data = np.load('MLProject2024/fashion_train.npy')
test_data = np.load('MLProject2024/fashion_test.npy')

train_images = train_data[:, :-1] 
train_labels = train_data[:, -1].astype(int)
test_images = test_data[:, :-1]
test_labels = test_data[:, -1].astype(int)

mean = np.mean(train_images, axis=0)
std = np.std(train_images, axis=0)
train_images = (train_images - mean) / std
test_images = (test_images - mean) / std 

#pca
pca = PCA(n_components=100)
train_images = pca.fit_transform(train_images)
test_images= pca.transform(test_images)

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# 1. Load and preprocess the data
train_data = np.load('MLProject2024/fashion_train.npy')
test_data = np.load('MLProject2024/fashion_test.npy')

train_images = train_data[:, :-1]
train_labels = train_data[:, -1].astype(int)
test_images = test_data[:, :-1]
test_labels = test_data[:, -1].astype(int)

# Normalize the data
mean = np.mean(train_images, axis=0)
std = np.std(train_images, axis=0)
train_images = (train_images - mean) / std
test_images = (test_images - mean) / std

# Reshape to match CNN input (N, C, H, W)
train_images = train_images.reshape(-1, 1, 28, 28)  # Assuming images are 28x28
test_images = test_images.reshape(-1, 1, 28, 28)

# Convert to PyTorch tensors
train_images_tensor = torch.tensor(train_images, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
test_images_tensor = torch.tensor(test_images, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)

# Create PyTorch datasets and dataloaders
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 2. Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)  # Conv layer 1
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # Conv layer 2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # Fully connected layer 1
        self.fc2 = nn.Linear(128, 10)  # Fully connected layer 2 (10 classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # Conv1 + ReLU + Pool
        x = self.pool(torch.relu(self.conv2(x)))  # Conv2 + ReLU + Pool
        x = x.view(-1, 64 * 7 * 7)  # Flatten for FC layers
        x = torch.relu(self.fc1(x))  # FC1 + ReLU
        x = self.fc2(x)  # FC2
        return x

# 3. Initialize the model, loss function, and optimizer
model = CNN()
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# 5. Testing loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Epoch 1/50, Loss: 0.5262932547936424
Epoch 2/50, Loss: 0.34795856249694246
Epoch 3/50, Loss: 0.29679346611848273
Epoch 4/50, Loss: 0.25668077220836766
Epoch 5/50, Loss: 0.22175772090594228
Epoch 6/50, Loss: 0.18964920461939547
Epoch 7/50, Loss: 0.15572421711545212
Epoch 8/50, Loss: 0.12555089896431745
Epoch 9/50, Loss: 0.10102342520528042
Epoch 10/50, Loss: 0.08385112015054391
Epoch 11/50, Loss: 0.06710335605230314
Epoch 12/50, Loss: 0.06885115145527113
Epoch 13/50, Loss: 0.03882975760176361
Epoch 14/50, Loss: 0.03653466082618798
Epoch 15/50, Loss: 0.023123098597579158
Epoch 16/50, Loss: 0.042775250607496165
Epoch 17/50, Loss: 0.031506514634531975
Epoch 18/50, Loss: 0.018877225097480077
Epoch 19/50, Loss: 0.013578901784185814
Epoch 20/50, Loss: 0.02986306242210833
Epoch 21/50, Loss: 0.019975262057315715
Epoch 22/50, Loss: 0.014548134743304813
Epoch 23/50, Loss: 0.02181551672215871
Epoch 24/50, Loss: 0.024839986307467577
Epoch 25/50, Loss: 0.028235335465598355
Epoch 26/50, Loss: 0.00786